In [1]:
# from splinter import Browser
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import pandas as pd

In [2]:
#path to chromedriver
executable_path = {'executable_path':'C:/chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [3]:
#visit the webpage
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
#scrape page into soup
html = browser.html
soup = bs(html, 'html.parser')

In [10]:
step1 = soup.find('ul', class_='item_list')
step2 = step1.find('li', class_='slide')
news_title = step2.find('div', class_='content_title').text
news_title

'MOXIE Could Help Future Rockets Launch Off Mars'

In [9]:
# Find paragraph text
news_p = step2.find('div', class_='article_teaser_body').text
news_p

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

## JPL Mars Page Space Images - Featured Image

In [11]:
#visit the webpage
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [16]:
# Click featured image
browser.links.find_by_partial_text('FULL IMAGE')

In [17]:
# Click again for full large image
time.sleep(3)
browser.links.find_by_partial_text('more info')

In [18]:
#create beautiful soup object
html = browser.html
soup = bs(html, 'html.parser')

In [19]:
# Search for image source
results = soup.find_all('figure', class_='lede')
relative_img_path = results[0].a['href']
featured_img_url = 'https://www.jpl.nasa.gov' + relative_img_path

print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA15253_hires.jpg


## Mars Facts

In [20]:
# Use Pandas to scrape data
table = pd.read_html('https://space-facts.com/mars/')

In [21]:
#use first table for mars facts only and name table columns
marsfacts_df = table[0]
marsfacts_df.columns = ["Characteristic", "Value"]
marsfacts_df

,Characteristic,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [32]:
# Convert table to html
marsfacts_html = marsfacts_df.to_html(classes='data table table-borderless', index=False, header=False, border=0)
marsfacts_html

'<table border="0" class="dataframe data table table-borderless">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

## Mars Hemispheres

In [23]:
#visit the webpage
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [24]:
#create beautiful soup object
html = browser.html
soup = bs(html, 'html.parser')

In [25]:
#create an empty list to append hemispheres to 
hemispheres = []

In [26]:
#find the hemisphere names
results = soup.find_all('div', class_="collapsible results")
hemis = results[0].find_all('h3')

In [27]:
# Get text and store in list
for name in hemis:
    hemispheres.append(name.text)

hemispheres

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [28]:
# Find the thumbnail images for each hemisphere
thumbnails = results[0].find_all('a')
thumbnail_link = []

In [29]:
#create for loop to extract thumnbail paths
for thumbnail in thumbnails:
    
    # If the thumbnail element has an image...
    if (thumbnail.img):
        
        # then grab the attached link
        thumbnail_path = 'https://astrogeology.usgs.gov/' + thumbnail['href']
        
        # Append list with links
        thumbnail_link.append(thumbnail_path)

thumbnail_link

['https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced']

In [30]:
#find the path to the full sized image for each hemisphere
tif_jpeg = []

for url in thumbnail_link:
    
    # Click through each thumbanil link
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Scrape each page for the relative image path
    results = soup.find_all('img', class_='wide-image')
    relative_img_path = results[0]['src']
    
    # Combine the reltaive image path to get the full url
    img_link = 'https://astrogeology.usgs.gov/' + relative_img_path
    
    # Add full image links to a list
    tif_jpeg.append(img_link)

tif_jpeg

['https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [31]:
#combine the two existing lists 
combined_hemi_jpeg = zip(hemispheres, tif_jpeg)

hemisphere_image_urls = []

# Iterate through the zipped object
for title, img in combined_hemi_jpeg:
    
    mars_dict = {}
    
    # Add hemisphere title to dictionary
    mars_dict['title'] = title
    
    # Add image url to dictionary
    mars_dict['img_url'] = img
    
    # Append the list with dictionaries
    hemisphere_image_urls.append(mars_dict)

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov//cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]